In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from proj1_helpers import *
#from implementations import *
DATA_FOLDER = '../data/'

In [3]:
from split_data import split_data

In [4]:
y_train, x_train, id_train = load_csv_data(DATA_FOLDER + 'train.csv')
y_test, x_test, id_test = load_csv_data(DATA_FOLDER + 'test.csv')

In [5]:
#x_train.filter(lambda v: v==v, x)

In [6]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [7]:
np.shape(x_train)

(250000, 30)

In [8]:
id_train

array([100000, 100001, 100002, ..., 349997, 349998, 349999])

### Basic Least Squares

In [5]:
def compare_prediction(w_train, x, y):
    y_pred = predict_labels(w_train, x)
    return (y_pred == y).sum()/len(y_pred)

In [10]:
w_ls, loss_ls = least_squares(y_train, x_train)
np.shape(w_ls)

(30,)

In [11]:
compare_prediction(w_ls, x_train, y_train)

0.74432799999999999

### Stochastic Gradient Descent

In [12]:
np.shape(np.asarray([1]*30)[:,None].reshape(30, 1))

(30, 1)

In [13]:
np.shape(np.asarray([1]*30))

(30,)

In [14]:
initial_w = np.asarray([1]*30)
max_iters = 30
gamma = 0.7

In [15]:
w_sgd, loss_sgd = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)

/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/loss.py:5: RuntimeWarning: overflow encountered in square
  return np.mean(e**2)/2.


In [16]:
compare_prediction(w_sgd, x_train, y_train)

0.62978400000000001

### Ridge Regression

In [69]:
def compute_loss(y, tx, w):
    e = y - tx @ w
    return np.mean(e**2)/2.

In [70]:
def ridge_regression(y, tx, lambda_):

    first_term = tx.T@tx
    #sum ici les x*w
    left = first_term + lambda_ *np.identity(tx.shape[1])
    right = tx.T @ y
    w = np.linalg.solve(left, right)
    loss = compute_loss(y, tx, w)
    return w, loss;

In [81]:
from build_polynomial import build_poly

def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    # split data
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    tr_poly = build_poly(x_tr, degree)
    te_poly = build_poly(x_te, degree)
    #print(np.shape(x_tr))
    #print(np.shape(tr_poly))
    #####
    #tr_poly = np.sum(np.split(tr_poly, degree+1, axis = 1), axis=0)
    #te_poly = np.sum(np.split(te_poly, degree+1, axis = 1), axis=0)
    #####
    
    #loss = 1000
    best_comp_pred = 0
    w = 0
    
    for lambda_ in lambdas:
        # ridge regression
        weight, l = ridge_regression(y_tr, tr_poly, lambda_)
        #weight = stack_w(weight, degree)
        mse_test = compute_loss(y_te, te_poly, weight)
        
        #print(np.shape(weight))
        #print(np.shape(tr_poly))
        #print(np.shape(y_tr))
        comp_pred = compare_prediction(weight.reshape(len(weight), 1), tr_poly, y_tr.reshape(len(y_tr), 1))
        if(comp_pred > best_comp_pred):
            best_comp_pred = comp_pred
            print(best_comp_pred)
            loss = mse_test
            w = weight
        
    return w, loss

In [139]:
# boucler sur les seeds pour avoir un meileur ? 
#seed = 95
#degree = 6
#split_ratio = 0.29

seed = 120
degree = 6
split_ratio = 0.29

w_r, loss_ridge = ridge_regression_demo(x_train, y_train, degree, split_ratio, seed)

0.799834482759
0.800744827586
0.800910344828


In [19]:
np.shape(y_train)

(250000,)

In [20]:
np.shape(w_r)

(210,)

In [21]:
w_s = np.sum(np.split(w_r, degree + 1),axis=0)
np.shape(w_s)

(30,)

In [34]:
np.shape(build_poly(x_train, degree))

(250000, 180)

In [140]:
compare_prediction(w_r.reshape(len(w_r), 1), build_poly(x_train, degree), y_train.reshape(len(y_train), 1))

0.80015999999999998

### Logistic Regression

In [24]:
def ridge_regression2(y, tx, lamb):
    """implement ridge regression."""
    aI = lamb * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

def ridge_regression3(y, tx, lambda_):
    first_term = tx.T.dot(tx)
    left = first_term + lambda_ *np.identity(tx.shape[1])
    right = tx.T.dot(y)
    w = np.linalg.solve(left, right)
    loss = compute_loss2(y, tx, w)
    return w, loss;

def compute_loss2(y, tx, w):
    e = y - tx.dot(w)
    return np.mean(e**2)/2.

### Bias-variance

In [25]:
from implementations import least_squares
from split_data import split_data

def bias_variance_demo():
    """The entry."""
    # define parameters
    seeds = range(100)
    num_data = 10000
    ratio_train = 0.005
    degrees = range(1, 10)
    lambdas = np.logspace(-5, 0, 15)
    
    best_seed = 0
    best_degree = 0
    best_lambda = 0
    min_loss = 1000
    for index_seed, seed in enumerate(seeds):
        np.random.seed(seed)
    
        # split data with a specific seed
        x_tr, y_tr, x_te, y_te = split_data(x_train, y_train, ratio_train, seed)
        
        # bias_variance_decomposition
        for index_deg, degree in enumerate(degrees):
            # form data with polynomial degree
            train_poly = build_poly(x_tr, degree)
            test_poly = build_poly(x_te, degree)
            
            w_train, loss_train = least_squares(y_tr, train_poly)

            # calculate the loss for train and test data
            loss_test = compute_loss(y_te, test_poly, w_train)
            
            if(loss_test < min_loss):
                min_loss = loss_test
                best_seed = seed
                #best_degree = degree
                best_lambda = lambda_
    #return best_seed, best_lambda, min_loss
    return best_seed, best_degree ,min_loss        


In [26]:
#s, d, l = bias_variance_demo()

In [27]:
#l

### Cross-validation

In [28]:
from build_k_indices import build_k_indices
from implementations import ridge_regression
from build_polynomial import build_poly

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    train_indices = [x for j, x in enumerate(k_indices) if j != k]
    train_indices = [idx for part in train_indices for idx in part]
    test_indices = k_indices[k]
    
    x_tr = x[train_indices]
    y_tr = y[train_indices]
    
    x_te = x[test_indices]
    y_te = y[test_indices]

    # form data with polynomial degree
    #train_poly = build_poly(x_tr, degree)
    #test_poly = build_poly(x_te, degree)

    # ridge regression
    
    weights_train, loss_tr = ridge_regression(y_tr, x_tr, lambda_)

    # calculate the loss for train and test data
    loss_te = compute_loss(y_te, x_te, weights_train)

    return loss_tr, loss_te

In [29]:
def cross_validation_demo():
    seed = 55
    degree = 2
    k_fold = 4
    lambdas = np.logspace(-4, 0, 30)
    
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    
    min_mean_loss = 1000
    best_lambdas = np.zeros(k_fold)
    # cross validation
    for i_test in range(k_fold):
        min_loss = 1000
        for lambda_ in lambdas:
            loss_tr, loss_te = cross_validation(y_train, x_train, k_indices, i_test, lambda_, degree)
            
            if(loss_te < min_loss):
                min_loss = loss_te
                best_lambdas[i_test] = lambda_
    lambda_mean = np.mean(best_lambdas)
    return lambda_mean, min_mean_loss

In [30]:
bl, mml = cross_validation_demo()

In [31]:
bl

0.25007499999999999

In [32]:
w, l = ridge_regression(y_train, x_train, bl)

In [33]:
compare_prediction(w, x_train, y_train)

0.74440799999999996

### Logistic Regression

In [6]:
def sigmoid(t):
    """apply sigmoid function on t."""
    sig = 2*(np.exp(t) / (1 + np.exp(t))) - 1
    #sig = (np.exp((t+1)/2) / (np.exp((t+1)/2)+1)
    return sig

In [7]:
def calculate_loss2(y, tx, w):
    """compute the cost by negative log likelihood."""
    pred = sigmoid(tx.dot(w))
    loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))
    return np.squeeze(- loss)

def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    tx_w = tx @ w
    
    loss = np.log(1 + np.exp(tx_w)) - (y * (tx_w))
    
    sum_loss = np.sum(loss)
    
    return sum_loss


In [8]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    
    return  tx.T @(sigmoid(tx @ w) - y)

In [9]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [10]:
def shuffle_index_resize(y,x, max_elem_percent):
    y_pos = []
    y_neg = []
    
    max_elem = int(((max_elem_percent * len(y) ) / 100) / 2.0)

    for idx_y, val in enumerate(y):

        if(val == 1):
            y_pos.append(idx_y)
        else:
            y_neg.append(idx_y)

        
    data_size_pos = len(y_pos)  
    shuffle_indices_pos = np.random.permutation(np.arange(data_size_pos)) 
    data_size_neg = len(y_neg)  
    shuffle_indices_neg = np.random.permutation(np.arange(data_size_neg)) 
    new_y = []
    new_x = []
    for i in range(max_elem):
        new_y.append(y[y_pos[shuffle_indices_pos[i]]])
        new_y.append(y[y_neg[shuffle_indices_neg[i]]])
        new_x.append(x[y_pos[shuffle_indices_pos[i]]])
        new_x.append(x[y_neg[shuffle_indices_neg[i]]])
    
    return np.array(new_y), np.array(new_x)


new_y, new_x = shuffle_index_resize(y_train, x_train, 0.2)

np.shape(new_x)

(500, 30)

In [11]:
def logistic_regression_gradient(y, x, w, gamma, max_iter):
    # init parameters
    #max_iter = 100000
    threshold = 1e-8
    #gamma = 0.000000001
    lambda_ = 0.02
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0], 1)), x]
    #w = np.zeros((tx.shape[1], 1))
    
    # start the logistic regression
    for iter_ in range(max_iter):
        # get loss and update w.
        
        #y, x = shuffle_index_resize2(y, x, 50000); #100000 = 40%  12500 = 5%
        
        if((iter_ % 1000) == 0):
            print(iter_)
        loss, w = learning_by_gradient_descent(y, x, w, gamma)
        
        #loss, w = learning_by_newton_method(y, tx, w)
        #loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
    return loss, w

#nb = 200

#x_train2 = x_train[:nb]

#y_train2 = y_train.reshape(len(y_train), 1)
#y_train2 = y_train2[:nb]
#loss, w = logistic_regression_gradient_descent_demo(y_train2, x_train2)
#loss

In [126]:
def test():
    #gammas = np.arange(0.48e-13,0.51e-13,1e-15)
    gammas = np.logspace(-15, 0, 30)
    # last = 5.5e-12

    max_iter = 5000
    #max_iter = 100000

    best_compare_pred = 0
    best_gamma = 0
    best_w = 0


    #y_train2, x_train2 = shuffle_index_resize(y_train,x_train, 68)
    x_train2 = x_train
    y_train2 = y_train

    y_train2 = y_train2.reshape(len(y_train2), 1)
    #print(np.shape(y_train2))
    #print(np.shape(x_train2))


    tx = np.c_[np.ones((y_train2.shape[0], 1)), x_train2]
    w_init = np.zeros((tx.shape[1], 1))

    #print(np.shape(w_init))

    gamma_step = len(gammas)

    for gamma_ in gammas:
    
        loss, w = logistic_regression_gradient(y_train2, tx, w_init, gamma_, max_iter)
    
        #print(np.shape(w))
        gamma_step -= 1;
        print('%d steps' % gamma_step)
        print(gamma_)
    
        compare_pred = compare_prediction(w, tx, y_train2)
        print('compare_pred  = ' + str(compare_pred) + ', best_compare_pred = '+ str(best_compare_pred))
      
        if(compare_pred > best_compare_pred):
            print('compare_pred = %f' % compare_pred)
            best_compare_pred = compare_pred;
            best_gamma = gamma_;
            best_w = w;
        
    return best_compare_pred, best_gamma, best_w 

0
1000
2000
3000
4000
29 steps
1e-15
compare_pred  = 0.640292, best_compare_pred = 0
compare_pred = 0.640292
0
1000
2000
3000
4000
28 steps
3.29034456231e-15
compare_pred  = 0.640596, best_compare_pred = 0.640292
compare_pred = 0.640596
0
1000
2000
3000
4000
27 steps
1.08263673387e-14
compare_pred  = 0.650876, best_compare_pred = 0.640596
compare_pred = 0.650876
0
1000
2000
3000
4000
26 steps
3.56224789026e-14
compare_pred  = 0.681028, best_compare_pred = 0.650876
compare_pred = 0.681028
0
1000
2000
3000
4000
25 steps
1.17210229753e-13
compare_pred  = 0.69362, best_compare_pred = 0.681028
compare_pred = 0.693620
0
1000
2000
3000
4000
24 steps
3.85662042116e-13
compare_pred  = 0.69798, best_compare_pred = 0.69362
compare_pred = 0.697980
0
1000
2000
3000
4000
23 steps
1.26896100317e-12
compare_pred  = 0.703372, best_compare_pred = 0.69798
compare_pred = 0.703372
0
1000
2000
3000
4000
22 steps
4.17531893656e-12
compare_pred  = 0.713932, best_compare_pred = 0.703372
compare_pred = 0.713932

/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()
/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


1000
2000
3000
4000
17 steps
1.61026202756e-09
compare_pred  = 0.0, best_compare_pred = 0.713932
0


/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/proj1_helpers.py:30: RuntimeWarning: invalid value encountered in less_equal
  y_pred[np.where(y_pred <= 0)] = -1
/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/proj1_helpers.py:31: RuntimeWarning: invalid value encountered in greater
  y_pred[np.where(y_pred > 0)] = 1


1000
2000
3000
4000
16 steps
5.29831690628e-09
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
15 steps
1.7433288222e-08
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
14 steps
5.73615251045e-08
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
13 steps
1.88739182214e-07
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
12 steps
6.21016941892e-07
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
11 steps
2.04335971786e-06
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
10 steps
6.7233575365e-06
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
9 steps
2.21221629107e-05
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
8 steps
7.27895384398e-05
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000
4000
7 steps
0.000239502661999
compare_pred  = 0.0, best_compare_pred = 0.713932
0
1000
2000
3000


10000

In [19]:
best_compare_pred, best_gamma, best_w  = test()
print(best_compare_pred)
print(best_gamma)
print(np.shape(best_w))

54.8999255836
4.9e-14


(31, 1)

In [20]:
compare_prediction(best_w, tx, y_train2)

0.69247599999999998

In [110]:
best_w_saved = best_w
best_gamma_saved = best_gamma
best_loss_saved = best_loss

In [101]:
np.shape(new_y)

(100000,)

In [58]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    # return loss, gradient, and hessian
    #loss = calculate_loss(y, tx, w) + (w.T@w*lambda_ / 2.0)
    
    grad = calculate_gradient(y, tx, w) + (w*lambda_)
    
    return grad

In [59]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    gradient = penalized_logistic_regression(y, tx, w, lambda_)
    w -= gamma * gradient
    return w

In [60]:
def shuffle_index_resize2(y,x, max_elem_percent):
    y_pos = []
    y_neg = []
    
    max_elem = int(max_elem_percent / 2)

    for idx_y, val in enumerate(y):

        if(val == 1):
            y_pos.append(idx_y)
        else:
            y_neg.append(idx_y)

        
    data_size_pos = len(y_pos)  
    shuffle_indices_pos = np.random.permutation(np.arange(data_size_pos)) 
    data_size_neg = len(y_neg)  
    shuffle_indices_neg = np.random.permutation(np.arange(data_size_neg)) 
    new_y = []
    new_x = []
    for i in range(max_elem):
        new_y.append(y[y_pos[shuffle_indices_pos[i]]])
        new_y.append(y[y_neg[shuffle_indices_neg[i]]])
        new_x.append(x[y_pos[shuffle_indices_pos[i]]])
        new_x.append(x[y_neg[shuffle_indices_neg[i]]])
    
    return np.array(new_y), np.array(new_x)


new_y, new_x = shuffle_index_resize2(y_train, x_train, 0.2)

np.shape(new_x)

(0,)

In [61]:
def logistic_regression_gradient_descent_demo(y, x, w, gamma, max_iter, lambda_):
    # init parameters
    #max_iter = 100000
    #gamma = 0.000000001
    #lambda_ = 0.02
    threshold = 1e-8
    losses = []
        
    # build tx
    #tx = np.c_[np.ones((y.shape[0], 1)), x]
    #w = np.zeros((tx.shape[1], 1))
    
    
    # start the logistic regression
    for iter_ in range(max_iter):
        # get loss and update w.
        
        y, x = shuffle_index_resize2(y, x, 50000); #100000 = 40%  12500 = 5%
        
        if((iter_ % 100) == 0):
            print(iter_)
        w = learning_by_penalized_gradient(y, x, w, gamma, lambda_)
        
        #losses.append(loss)
        #if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            #break
        
        #loss, w = learning_by_newton_method(y, tx, w)
        #loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
    return w




In [62]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    
    # problem ici avec 1 et sum
    phi_x = []
    
    for j in range(0, degree + 1):
        if(j == 0):
            phi_x = np.power(x, j)
        else:
            x_power_j = np.power(x, j)
            phi_x = np.c_[phi_x, x_power_j]

    return phi_x

In [124]:
gammas = np.logspace(-15, -10, 5)
#gammas = np.arange(0.48e-13,1.0e-13,1e-15)
# last = 7.19685673001e-12  1e-13

max_iter = 400
#max_iter = 100000

lambdas = np.logspace(-20, -15, 5)
#lambdas = np.logspace(-13, -10, 2)
#lambdas = np.arange(1.9e-9,2.1e-9,1e-10)
# last =

best_comp_pred = 0
best_gamma = 0
best_w = 0
best_lambda = 0


#degree = 4
#x_train2 = build_poly(x_train, degree)
x_train2 = x_train
y_train2 = y_train
#y_train2, x_train2 = shuffle_index_resize(y_train,x_train, 60)

y_train2 = y_train2.reshape(len(y_train2), 1)

tx = np.c_[np.ones((y_train2.shape[0], 1)), x_train2]
w_init = np.zeros((tx.shape[1], 1))

gamma_step = len(gammas)

for gamma_ in gammas:
    lambda_step = len(lambdas)
    
    for lambda_ in lambdas:
        w = logistic_regression_gradient_descent_demo(y_train2, tx, w_init, gamma_, max_iter, lambda_)
        
        #comp_pred = compare_prediction(weight.reshape(len(weight), 1), tr_poly, y_tr.reshape(len(y_tr), 1))
        
        #loss, w = logistic_regression_gradient_descent_demo(y_train2, tx, w_init, gamma_, max_iter, lambda_)
        print('degree')
        #compare_prediction(w_r.reshape(len(w_r), 1), build_poly(x_train, degree), y_train.reshape(len(y_train), 1))

        comp_pred = compare_prediction(w, tx, y_train2)
        print('comp_pred  = ' + str(comp_pred) + ', best_comp_pred = '+ str(best_comp_pred))
        
        if(comp_pred > best_comp_pred):
            print('comp_pred = %f' % comp_pred)
            best_comp_pred = comp_pred;
            best_gamma = gamma_;
            best_w = w;
            best_lambda = lambda_
            
        lambda_step -= 1;
        print('%d steps_lambda' % lambda_step)
        print('lambda')
        print(lambda_)
        
        #comp_pred = compare_prediction(w, tx, y_train2)
        #print('comp_pred  = ' + str(comp_pred) + ', best_comp_pred = '+ str(best_comp_pred))
        
        #if(comp_pred > best_comp_pred):
            #print('comp_pred = %f' % comp_pred)
            #best_comp_pred = comp_pred;
            #best_gamma = gamma_;
            #best_w = w;
            #best_lambda = lambda_
    
    #print(np.shape(w))
    gamma_step -= 1;
    print('%d steps_gamma' % gamma_step)
    print('gamma')
    print(gamma_)
        
best_loss    

0
100
200


KeyboardInterrupt: 

In [ ]:
print(best_loss)
print(best_gamma)
np.shape(best_w)

In [39]:
compare_prediction(best_w, tx, y_train2)

0.68149999999999999

In [ ]:
best_w2_saved = best_w
best_gamma2_saved = best_gamma
best_loss2_saved = best_loss

In [17]:
def cross_validation(y, x, k_indices, k, lambda_, gamma_, max_iter):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    train_indices = [x for j, x in enumerate(k_indices) if j != k]
    train_indices = [idx for part in train_indices for idx in part]
    test_indices = k_indices[k]
    
    x_train = x[train_indices]
    y_train = y[train_indices]
    
    y_train2 = y_train.reshape(len(y_train), 1)

    x_train2 = np.c_[np.ones((y_train2.shape[0], 1)), x_train]
    w_init = np.zeros((tx.shape[1], 1))
    
    # Logistic regression
    
    #print(np.shape(y_train2))
    #print(np.shape(x_train2))
    
    #weights_train, loss_tr = ridge_regression(y_train, train_poly, lambda_)
    loss_tr, weights_train = logistic_regression_gradient_descent_demo(y_train2, x_train2, w_init, gamma_, max_iter, lambda_)
    
    return loss_tr

In [18]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [25]:
def cross_validation_demo():
    seed = 1
    k_fold = 2
    max_iter = 400
    lambdas = np.logspace(-13, 0, 15)
    gammas = np.logspace(-13, 0, 15)
    
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    
    best_loss_train = 100000
    best_gamma = 0
    best_lambda = 0
    #best_w = 0
    
    gamma_step = len(gammas)
    
    # cross validation
    for gamma_ in gammas:
        loss_tr_tmp = []
        lambda_step = len(lambdas)
        
        for lambda_ in lambdas: 
            
            for i_test in range(k_fold): 
                loss_tr = cross_validation(y_train, x_train, k_indices, i_test, lambda_, gamma_, max_iter)
                loss_tr_tmp.append(loss_tr)
                print('loss  = ' + str(loss) + ', best_loss = '+ str(best_loss_train))
            
            lambda_step -= 1;
            print('%d steps_lambda' % lambda_step)
            print('lambda')
            print(lambda_)
            print('loss_mean')
            print(np.sqrt(2*np.mean(loss_tr_tmp)))
            
            #compare_pred = compare_prediction(best_w, tx, y_train2)
            
            loss_train = np.sqrt(2*np.mean(loss_tr_tmp))
            
            if(loss_train < best_loss_train):
                
                best_loss_train = loss_train
                best_gamma = gamma_
                best_lambda = lambda_
                #best_w = w  
        gamma_step -= 1;
        print('%d steps_gamma' % gamma_step)
        print('gamma')
        print(gamma_)

cross_validation_demo()

0
100
200
300
loss  = 29503.2235826, best_loss = 10000
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
14 steps_lambda
lambda
1e-13
lambda_mean
241.911853099
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
13 steps_lambda
lambda
8.48342898244e-13
lambda_mean
241.339107838
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
12 steps_lambda
lambda
7.19685673001e-12
lambda_mean
241.135487565
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
11 steps_lambda
lambda
6.10540229659e-11
lambda_mean
241.121210965
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
10 steps_lambda
lambda
5.17947467923e-10
lambda_mean
241.271827555
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
0
100
200
300
loss  = 29503.2235826, best_loss = 10000
9 steps_lambda
la

KeyboardInterrupt: 

In [46]:
print(best_loss)
print(best_gamma)
np.shape(best_w)

array([  1.00000000e-13,   1.66088278e-13,   2.75853162e-13,
         4.58159767e-13,   7.60949669e-13,   1.26384820e-12,
         2.09910372e-12,   3.48636523e-12,   5.79044398e-12,
         9.61724871e-12,   1.59731228e-11,   2.65294846e-11,
         4.40623643e-11,   7.31824222e-11,   1.21547425e-10,
         2.01876025e-10,   3.35292415e-10,   5.56881399e-10,
         9.24914728e-10,   1.53617495e-09,   2.55140652e-09,
         4.23758716e-09,   7.03813555e-09,   1.16895182e-08,
         1.94149195e-08,   3.22459055e-08,   5.35566692e-08,
         8.89513497e-08,   1.47737765e-07,   2.45375111e-07,
         4.07539297e-07,   6.76875001e-07,   1.12421004e-06,
         1.86718109e-06,   3.10116893e-06,   5.15067808e-06,
         8.55467254e-06,   1.42083083e-05,   2.35983347e-05,
         3.91940677e-05,   6.50967523e-05,   1.08118075e-04,
         1.79571449e-04,   2.98247129e-04,   4.95353521e-04,
         8.22724134e-04,   1.36644835e-03,   2.26951054e-03,
         3.76939098e-03,

In [ ]:
y_train2 = y_train.reshape(len(y_train), 1)
tx = np.c_[np.ones((y_train2.shape[0], 1)), x_train]

compare_prediction(best_w, tx, y_train2)